## Import packages

In [108]:
import selenium
from selenium import webdriver

import shutil, time, os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from importlib import reload

import src
reload(src)

<module 'src' from 'C:\\Users\\Hugh\\git\\warcraftLogs\\src.py'>

## Configurations

In [39]:
# Set path variables
path_to_adblocker = r"C:\Users\Hugh\Desktop\3.11.2_0"
path_to_download_dir = r"C:\Users\Hugh\Downloads"

In [107]:
# Number of parses to scrape (Current max is 100 or 1 page)
N_parses = 21

## Navigate to WCL

In [41]:
boss = "High Warlord Naj'entus"

In [42]:
# Dictionary for browser to navigate to boss page URLs
boss_link_dict = {"High Warlord Naj'entus" : "#boss=601", "Supremus" : "#boss=602", "Shade of Akama" : "#boss=603", 
                  "Teron Gorefiend" : "#boss=604", "Gurtogg Bloodboil" : "#boss=605", "Reliquary of Souls" : "#boss=606", 
                  "Mother Shahraz" : "#boss=607", "The Illidari Council" : "#boss=608", "Illidan Stormrage" : "#boss=609", 
                  "Rage Winterchill" : "#boss=618", "Anetheron" : "#boss=619", "Kaz'rogal" : "#boss=620", 
                  "Azgalor" : "#boss=621", "Archimonde" : "#boss=622"}

In [43]:
boss_page_url, browser = src.load_top_N_scraper(path_to_adblocker, boss, boss_link_dict)

## Main code loop

In [ ]:
def main():
    
    # First check to see if any of the rankings changed since last scraping and update the excel file
    print("Checking for rank changes since last scrape...")
    for i in range(1, N_parses):
        rank, name, server, region, date, HPS, duration = src.get_boss_data_top_N_scraper(browser, boss, boss_link_dict, i)
        
        rank_changed = src.check_if_rank_changed(boss, rank, name, date)
        if rank_changed: 
            src.update_rank(boss, rank, name, date)
    print("Rank updates complete.")
    time.sleep(2)
    
    
    print("Beginning data scrape...")
    for i in range(1, N_parses):
        
        rank, name, server, region, date, HPS, duration = src.get_boss_data_top_N_scraper(browser, boss, boss_link_dict, i)
        
        if src.check_if_parse_already_recorded_top_N(boss, rank, name): 
                continue
                
        print(rank, name, server, region)
    
        link = browser.find_element_by_link_text(name)
        link.click()
        time.sleep(1)
    
        player_df = pd.DataFrame(pd.np.empty((0, 23)))
        player_df.columns = ["Rank", "Name", "Server", "Date", "Duration", "nHealers", "Spriest?", "Innervate?", "Bloodlust?", "Nature's Grace?", "Power Infusion?", "LB_uptime", "HPS", "% LB (tick) HPS", "% LB (bloom) HPS", "% Rejuv HPS", "% Regrowth HPS", "% Swiftmend HPS", "Rotating on tank?", "Rotation 1", "% Rotation 1", "Rotation 2", "% Rotation 2"]
    
        temp_url = browser.current_url
        time.sleep(1)
        
        boss_tanks = src.get_tanks(browser)
        nHealers = src.get_nHealers(browser)
    
        print(f"Tanks: {boss_tanks}")
        
        browser.get(temp_url)
        time.sleep(0.5)
        
        player_link = browser.find_element_by_link_text(name)
        player_link.click()
        time.sleep(0.5)
        
        # Scrape spell HPS
        LBtick_HPS, LBbloom_HPS, rejuv_HPS, regrowth_HPS, swiftmend_HPS, LB_uptime = src.get_spell_info(browser, HPS)
        time.sleep(2)
                   
        # Scrape mana sources
        spriest = src.check_spriest(browser)
        innervate, bloodlust, powerInfusion, naturesGrace = src.check_buffs(browser)
        time.sleep(0.5)    
        
        # Download the cast-sequence CSV.
        src.download_csv(browser, temp_url, "filter-casts-tab", path_to_download_dir, "data/cast_sequence.csv")
        time.sleep(1)
        
        # Clean the csv
        df = src.clean_cast_sequence_csv()
        df = src.fix_cast_time(df)
        time.sleep(0.5)
        
        # Get the rotations
        rotation1, rotation1_percent, rotation2, rotation2_percent, rotating_on_tank = src.calculate_rotations(df, boss, boss_tanks)
        print(f'Rotations: {rotation1} ({rotation1_percent}), {rotation2} ({rotation2_percent})')  

        to_append = [rank, name, server + " " + region, date, duration, str(nHealers), spriest, innervate, bloodlust, powerInfusion, naturesGrace, LB_uptime, HPS, LBtick_HPS, LBbloom_HPS, rejuv_HPS, regrowth_HPS, swiftmend_HPS, rotating_on_tank, rotation1, rotation1_percent, rotation2, rotation2_percent]
        src.export_to_excel(boss, to_append, player_df, name, 'top_N_druids', True)
        
        os.remove(f"data/{boss.replace(' ', '')}_{name}.csv")
        os.remove(f"data/cast_sequence.csv")
        os.remove(f"data/cast_sequence_fixed.csv")
        
        print('-------')
        browser.get(boss_page_url)
        time.sleep(1)
        
        
    print(f"{boss} scraping complete.")
    print("---------------")
    
    
if __name__ == "__main__":
    main()

Checking for rank changes since last scrape...
Rank updates complete.
Beginning data scrape...
3 搞不得 末日之刃 (CN)
Tanks: ['硬抗', '一盾暴揍', '白胖胖不胖']
Rotations: 0LB 4RG (0.609), 0LB 3RG 1I (0.217)
-------
4 喵绿喵绿喵  范沃森
Tanks: ['唐婉', '阿斯兰']
Rotations: 0LB 4RG (0.591), 0LB 3RG 1I (0.318)
-------
5 Дарнейм  Пламегор
Tanks: ['Гунглай', 'Кальциум']
Rotations: 0LB 4I (1.0), None (0.0)
-------
6 芬巴巴之花  碧空之歌
Tanks: ['虎突突', '酸奶溶豆', '侟在']
Rotations: 0LB 4RG (0.56), 0LB 3RG 1I (0.32)
-------
7 黑夜现苍鹰 伦鲁迪洛尔 (CN)
Tanks: ['锦缕衣', '虎牙直播厅长']


In [105]:
browser.get(boss_page_url)